# Actividad 1 – Big Data  
## Creación de una Base de Datos Analítica para la empresa Clínica del Play S.A.S.  
### Estudiante: Justin Beckham Cardona Yepes  
### Materia: Big Data  
### Institución: IU Digital de Antioquia  

# 1. Definición del Problema

La empresa **Clínica del Play S.A.S.** se dedica al análisis, diagnóstico y optimización de ventas de videojuegos exclusivos de consolas PlayStation.  
Actualmente trabaja con tiendas de videojuegos, distribuidores minoristas y desarrolladores independientes.

La empresa enfrenta un problema:  
No posee una **base de datos analítica unificada** que permita realizar consultas estratégicas sobre:

- Juegos más vendidos.  
- Comparación de ventas entre PS3, PS4 y PS5.  
- Tendencias por género.  
- Desempeño por editora (Publisher).  
- Variación de ventas entre regiones del mundo.

Para resolver esta necesidad, se desarrollará una base de datos analítica basada en un dataset real que permitirá tomar decisiones basadas en datos.

Este proyecto construirá:

✅ Un modelo entidad–relación  
✅ Base de datos analítica  
✅ Carga del dataset  
✅ Consultas SQL  
✅ Documentación completa en Jupyter Notebook  


# 2. Dataset Seleccionado

Se utiliza el siguiente dataset de Kaggle:

**PlayStation Sales and Metadata (PS3, PS4, PS5)**  
Enlace: https://www.kaggle.com/datasets/gvidalguiresse/playstation-sales-and-metadata-ps3ps4ps5/data

## Motivo de selección

Este dataset contiene información esencial para la operación de Clínica del Play S.A.S., como:

- Nombre del videojuego  
- Plataforma (PS3, PS4, PS5)  
- Género  
- Editora  
- Año de lanzamiento  
- Ventas por región  
- Ventas globales  

## Variables relevantes

| Variable | Descripción | Utilidad |
|---------|-------------|----------|
| Name | Nombre del juego | Identificación |
| Platform | PS3/PS4/PS5 | Comparación entre generaciones |
| Genre | Género del juego | Identificar tendencias |
| Publisher | Empresa editora | Evaluar desempeño por marca |
| Year_of_Release | Año de lanzamiento | Análisis temporal |
| Global_Sales | Ventas totales | Indicador principal |
| NA_Sales, EU_Sales, JP_Sales | Ventas por región | Segmentación geográfica |

Estas variables permiten análisis completos para recomendaciones comerciales y de mercado.


## 🧩 Modelo Entidad-Relación

El modelo entidad-relación propuesto para la empresa **Clínica del Play S.A.S.** tiene como objetivo estructurar una base de datos analítica que permita gestionar y analizar la información de las ventas de videojuegos de manera eficiente.  
El modelo está compuesto por **tres entidades principales**: **Desarrolladora**, **Juego** y **Venta**, las cuales se relacionan entre sí para reflejar la realidad del negocio.

---

### **1. Entidad: Desarrolladora**
Representa a las empresas encargadas de crear y publicar los videojuegos.

**Atributos:**
- `id_desarrolladora` (PK): Identificador único de la desarrolladora.  
- `nombre`: Nombre de la empresa desarrolladora o publisher.  
- `pais`: País de origen (puede ser nulo o estimado).  
- `anio_fundacion`: Año en el que fue fundada la empresa.

---

### **2. Entidad: Juego**
Contiene la información específica de cada videojuego disponible en el mercado.

**Atributos:**
- `id_juego` (PK): Identificador único del videojuego.  
- `titulo`: Nombre del videojuego.  
- `genero`: Categoría o tipo del juego (acción, deportes, aventura, etc.).  
- `plataforma`: Consola o sistema en el que se lanzó (PS4, PS5, Xbox, etc.).  
- `fecha_lanzamiento`: Fecha aproximada de lanzamiento.  
- `id_desarrolladora` (FK): Relación con la entidad **Desarrolladora**, indicando qué empresa lo desarrolló.

**Relación:**  
Una **Desarrolladora puede crear varios Juegos**, pero **cada Juego pertenece a una sola Desarrolladora**.  
➡️ **Cardinalidad:** 1 a N (Desarrolladora → Juego)

---

### **3. Entidad: Venta**
Registra los datos de ventas de cada videojuego por región o a nivel global.

**Atributos:**
- `id_venta` (PK): Identificador único de la venta.  
- `id_juego` (FK): Referencia al videojuego vendido.  
- `region`: Región donde se realizó la venta (ej. NA, EU, JP, Other o Global).  
- `anio`: Año en que se registró la venta.  
- `unidades_vendidas`: Número de unidades vendidas (en millones).  
- `ingresos`: Total de ingresos generados (en millones de dólares o equivalente).

**Relación:**  
Un **Juego puede tener múltiples registros de Venta** (por distintas regiones o años), pero **cada Venta corresponde a un solo Juego**.  
➡️ **Cardinalidad:** 1 a N (Juego → Venta)

---

### **📈 Relaciones Finales del Modelo**
- **Desarrolladora (1) → (N) Juego**  
- **Juego (1) → (N) Venta**

---

### **💡 Beneficios del Modelo**
- Permite realizar análisis detallados de ventas por **juego, región, género o plataforma**.  
- Facilita identificar las **desarrolladoras con mejor rendimiento comercial**.  
- Proporciona una estructura flexible para **consultas analíticas** y **dashboards** en herramientas como Power BI o SQL.

### Diagrama ERD – Clínica del Play

A continuación se muestra el modelo entidad–relación diseñado para la base de datos analítica de la Clínica del Play:

![ERD Clínica del Play](./MODELOER.png)



## 4. Creación y carga de la base de datos en Databricks

En esta sección se implementa la base de datos analítica de la **Clínica del Play S.A.S.**,  
siguiendo el modelo entidad–relación previamente diseñado.

Se utiliza **Apache Spark SQL** dentro de **Databricks Community Edition** para:

1. Crear la base de datos `clinica_play_db`.
2. Crear las tres tablas: `desarrolladora`, `juego` y `venta`.
3. Cargar los datos del dataset `playdataset.csv` (descargado desde Kaggle).
4. Mostrar las primeras filas de cada tabla para validar la carga.

El objetivo es disponer de una base de datos analítica que permita realizar consultas
posteriores sobre las ventas de videojuegos, las desarrolladoras y sus plataformas.

# Crear base de datos

